In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import pytz
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

try:
  # Use the %tensorflow_version magic if in colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
X_train = pd.read_csv('../datasets/scaled_train_test/X_train_standard.csv', delimiter=',')
X_test = pd.read_csv('../datasets/scaled_train_test/X_test_standard.csv', delimiter=',')
y_train = X_train['Value (kWh)']
y_test = X_test['Value (kWh)']
X_train.drop(['Value (kWh)'], axis=1, inplace=True)
X_test.drop(['Value (kWh)'], axis=1, inplace=True)

In [73]:
X_test.columns

Index(['Pressure (msl) (hPa)', 'Relative humidity (%)',
       'Air temperature (degC)', 'Dew-point temperature (degC)',
       'Wind speed (m/s)', 'Global radiation (W/m2)', 'Wind_direction',
       'Cloud_amount_binary', 'Precipitation_binary', 'hour_of_day',
       'day_of_year', 'solar_elev_angle', 'sun_azimuth'],
      dtype='object')

In [3]:
import tensorflow as tf
keras = tf.keras
keras.backend.clear_session()
# tf.random.set_seed(42)
np.random.seed(42)

C:\Users\screwdriver\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\screwdriver\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\screwdriver\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\screwdriver\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:5

In [4]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

We will split the X_train data further into X_train and X_val, which is going to be our validation set.

In [5]:
from sklearn.model_selection import train_test_split

In [22]:
X_train_v, X_val, y_train_v, y_val = train_test_split(X_train, y_train, random_state=42, test_size=0.8) 

In [23]:
nn = keras.models.Sequential()
nn.add(keras.layers.Dense(30, activation='relu', input_shape=X_train_v.shape[1:]))
nn.add(keras.layers.Dense(1))
lr_schedule = keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch / 20))
reset_states = ResetStatesCallback()
optimizer = keras.optimizers.SGD(lr=1e-8, momentum=0.9)
nn.compile(optimizer=optimizer, loss='mean_squared_error')

AttributeError: module 'tensorflow.tools.api.generator.api.keras.losses' has no attribute 'Huber'

In [84]:
y_train_v.shape

(6759,)

In [13]:
history = nn.fit(X_train_v,y_train_v, epochs=100,
                 validation_data=(X_val,y_val),
                callbacks=[lr_schedule, reset_states])

Train on 6759 samples, validate on 27037 samples
Epoch 1/100
6759/6759 [==============================] - 1s 102us/step - loss: 1.3797 - val_loss: 1.3657
Epoch 2/100
6759/6759 [==============================] - 0s 65us/step - loss: 1.3794 - val_loss: 1.3654
Epoch 3/100
6759/6759 [==============================] - 0s 70us/step - loss: 1.3790 - val_loss: 1.3651
Epoch 4/100
6759/6759 [==============================] - 0s 74us/step - loss: 1.3787 - val_loss: 1.3647
Epoch 5/100
6759/6759 [==============================] - 0s 66us/step - loss: 1.3783 - val_loss: 1.3643
Epoch 6/100
6759/6759 [==============================] - 0s 67us/step - loss: 1.3778 - val_loss: 1.3638
Epoch 7/100
6759/6759 [==============================] - 0s 64us/step - loss: 1.3773 - val_loss: 1.3633
Epoch 8/100
6759/6759 [==============================] - 0s 69us/step - loss: 1.3767 - val_loss: 1.3627
Epoch 9/100
6759/6759 [==============================] - 0s 74us/step - loss: 1.3761 - val_loss: 1.3620
Epoch 10/100
6

6759/6759 [==============================] - 1s 96us/step - loss: 0.1750 - val_loss: 0.1726
Epoch 79/100
6759/6759 [==============================] - 1s 80us/step - loss: 0.1688 - val_loss: 0.1664
Epoch 80/100
6759/6759 [==============================] - 0s 66us/step - loss: 0.1629 - val_loss: 0.1606
Epoch 81/100
6759/6759 [==============================] - 0s 73us/step - loss: 0.1574 - val_loss: 0.1553
Epoch 82/100
6759/6759 [==============================] - 1s 91us/step - loss: 0.1523 - val_loss: 0.1500
Epoch 83/100
6759/6759 [==============================] - 0s 72us/step - loss: 0.1475 - val_loss: 0.1453
Epoch 84/100
6759/6759 [==============================] - 0s 70us/step - loss: 0.1431 - val_loss: 0.1408
Epoch 85/100
6759/6759 [==============================] - 0s 71us/step - loss: 0.1388 - val_loss: 0.1366
Epoch 86/100
6759/6759 [==============================] - 0s 73us/step - loss: 0.1349 - val_loss: 0.1330
Epoch 87/100
6759/6759 [==============================] - 1s 75us/st

Read the proper learning rate, typically at the elbow of the graph.

In [17]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 1e-3, 0,10])

KeyError: 'lr'

In [21]:
history.history.keys()

dict_keys(['val_loss', 'loss'])

In [ ]:
Selected learning rate: __FILL__

#### Training NN model
The model will be training for maximum 500 epochs, saving itself each time it improves in __'my_checkpoint.h5'__. If there is no more improvement for the next 50 epochs, the __EarlyStopping__ callback will finish the training and use the checkpointed best model. 

In [ ]:
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.LSTM(64,
                                    return_sequences=True,
                                    input_shape=x_train_multi.shape[-2:]))
nn_model.add(tf.keras.layers.LSTM(32, return_sequences=True, activation='relu'))
nn_model.add(tf.keras.layers.LSTM(16, return_sequences=False, activation='relu'))
nn_model.add(tf.keras.layers.Dense(future_target))
nn_model.add(tf.keras.layers.Lambda(lambda x: x * 200.0))
optimizer = keras.optimizers.SGD(lr=0.5e-5, momentum=0.9)
nn_model.compile(optimizer=optimizer, loss=keras.losses.Huber(), metrics=['mae'])
reset_states = ResetStatesCallback()
model_checkpoint = keras.callbacks.ModelCheckpoint(
    "my_checkpoint.h5", save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=50)
nn_model.fit(train_data_multi, epochs=500, steps_per_epoch=200,
                                          validation_data=val_data_multi,
                                          validation_steps=50, callbacks=[early_stopping, model_checkpoint, reset_states])


In [ ]:
model = keras.models.load_model("my_checkpoint.h5")